# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "../pandas-resources/schools_complete.csv"
student_data_to_load = "../pandas-resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
pycityschools = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
pycityschoolsdata = pycityschools.copy()
#change data types for numeric valuse that will be analyzed. Div does not work with str or int values.
pycityschools['reading_score'] = pycityschools['reading_score'].astype('float').map('{:.0f}'.format)
pycityschools['math_score'] = pycityschools['math_score'].astype('float').map('{:.0f}'.format)
pycityschools['size'] = pycityschools['size'].astype('float').map('{:,.0f}'.format)
pycityschools['budget'] = pycityschools['budget'].astype('float').map('${:,.2f}'.format)
# 
#pycityschools.head()
pycityschools.tail()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,"1,635","$1,043,130.00"
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,"1,635","$1,043,130.00"
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,"1,635","$1,043,130.00"
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,"1,635","$1,043,130.00"
39169,39169,Carolyn Jackson,F,11th,Thomas High School,95,75,14,Charter,"1,635","$1,043,130.00"


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:

schoolcount = pycityschools['School ID'].nunique()
studentcount = pycityschools['Student ID'].nunique()

totalbudget = '${:,.2f}'.format(school_data['budget'].sum())

totalavgmath = '{:,.2f}'.format(student_data['math_score'].mean())
totalavgreading = '{:,.2f}'.format(student_data['reading_score'].mean())

overallpassrate = '{:,.2f}'.format((float(totalavgmath) + float(totalavgreading))/2)

passmathcount = student_data.loc[student_data['math_score'] >= 70]['math_score'].count()
percentpassmath = '{:,.1f}%'.format(passmathcount / studentcount * 100)

passreadcount = student_data.loc[student_data['reading_score'] >= 70]['reading_score'].count()
percentpassread = '{:,.1f}%'.format(passreadcount / studentcount * 100)

schoolscorestats = pd.DataFrame({"Number of Schools":[schoolcount], "Number of Students":[studentcount],
                              "Total Budget":[totalbudget], "Avg Math Score":[totalavgmath],
                                "Avg Reading Score":[totalavgreading],"Overall Passing Rate":[overallpassrate],
                                 "% Passing Math Score":[percentpassmath],"% Passing Reading Score":[percentpassread]
                               },index=['Stats'])

schoolscorestats['Number of Students'] = schoolscorestats['Number of Students'].map('{:,.0f}'.format)

schoolscorestats=schoolscorestats.transpose()

schoolscorestats


,Stats
Number of Schools,15
Number of Students,"39,170"
Total Budget,"$24,649,428.00"
Avg Math Score,78.99
Avg Reading Score,81.88
Overall Passing Rate,80.44
% Passing Math Score,75.0%
% Passing Reading Score,85.8%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [3]:


pycityschoolstats = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

#set data types
pycityschoolstats['math_score'] = pycityschoolstats['math_score'].astype('float')
pycityschoolstats['reading_score'] = pycityschoolstats['reading_score'].astype('float')
pycityschoolstats['budget'] = pycityschoolstats['budget'].astype('float')

# collect needed values grouped by school
schooltype = pycityschoolstats.groupby(['school_name']).max()["type"]
schoolstudtotal = pycityschoolstats.groupby(['school_name']).count()["Student ID"]

# total school budget by school_name - using .mean which is sum of the budget devided by count. 
schooltotalbudget = pycityschoolstats.groupby(['school_name']).mean()["budget"]

schoolperstudbudget = schooltotalbudget / schoolstudtotal

schoolavgmathscore = pycityschoolstats.groupby(['school_name']).mean()["math_score"]

schoolavgreadscore = pycityschoolstats.groupby(['school_name']).mean()["reading_score"]

schoolpassmath = pycityschoolstats.loc[pycityschoolstats["math_score"] >=70].groupby(['school_name']).count()["math_score"]
schoolpassmathrate = schoolpassmath / schoolstudtotal *100

schoolpassread = pycityschoolstats.loc[pycityschoolstats["reading_score"] >=70].groupby(['school_name']).count()["reading_score"]
schoolpassreadrate = schoolpassread / schoolstudtotal *100

schoolallpassrate = (schoolpassmathrate + schoolpassreadrate)/2

#create sumary DataFrame
schoolsummaryfile = pd.DataFrame({"Type":schooltype, "Total Students":schoolstudtotal, "Total School Budget":schooltotalbudget, 
                                  "Per Student Budget":schoolperstudbudget, "Average Math Score":schoolavgmathscore, 
                                  "Average Reading Score":schoolavgreadscore, "% Passing Math":schoolpassmathrate,
                                  "% Passing Reading":schoolpassreadrate, "% Overall Passing Rate":schoolallpassrate})



#format DataFrame values
schoolsummarydata = schoolsummaryfile.copy()
#clean up index name
schoolsummarydata = schoolsummarydata.rename_axis("School Name")
schoolsummarydata['Total Students'] = schoolsummarydata['Total Students'].astype(float).map('{:,.0f}'.format)
schoolsummarydata['Total School Budget'] = schoolsummarydata['Total School Budget'].astype(float).map('${:,.2f}'.format)
schoolsummarydata['Per Student Budget'] = schoolsummarydata['Per Student Budget'].astype(float).map('${:,.2f}'.format)
schoolsummarydata['Average Math Score'] = schoolsummarydata['Average Math Score'].astype(float).map('{:.2f}'.format)
schoolsummarydata['Average Reading Score'] = schoolsummarydata['Average Reading Score'].astype(float).map('{:.2f}'.format)
schoolsummarydata['% Passing Math'] = schoolsummarydata['% Passing Math'].astype(float).map('{:.2f}%'.format)
schoolsummarydata['% Passing Reading'] = schoolsummarydata['% Passing Reading'].astype(float).map('{:.2f}%'.format)
schoolsummarydata['% Overall Passing Rate'] = schoolsummarydata['% Overall Passing Rate'].astype(float).map('{:.2f}%'.format)



schoolsummarydata


,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,74.31%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,73.80%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,73.81%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,94.38%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,73.64%


In [4]:
schooltop5 = schoolsummarydata.sort_values("% Overall Passing Rate", ascending=False)
schooltop5.head(5)

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,95.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,95.27%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,95.20%


* Sort and display the top five schools in overall passing rate

## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [5]:
schoolbottom5 = schoolsummarydata.sort_values("% Overall Passing Rate", ascending=False)
schoolbottom5.tail(5)

,Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,73.80%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,73.64%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,73.29%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
# collect data by grade 
ninethgrade = pycityschoolstats.loc[pycityschoolstats["grade"] == "9th", :]
tenthgrade = pycityschoolstats.loc[pycityschoolstats["grade"] == "10th", :]
eleventhgrade = pycityschoolstats.loc[pycityschoolstats["grade"] == "11th", :]
twelvethgrade = pycityschoolstats.loc[pycityschoolstats["grade"] == "12th", :]

# collect average scores for each grade grouped by school_name
ninethgrademathscore = ninethgrade.groupby(['school_name']).mean()["math_score"]
tenthgrademathscore = tenthgrade.groupby(['school_name']).mean()["math_score"]
eleventhgrademathscore = eleventhgrade.groupby(['school_name']).mean()["math_score"]
twelvethgrademathscore = twelvethgrade.groupby(['school_name']).mean()["math_score"]

# create DataFrame for summary data
mathsummarydata = pd.DataFrame({"Avg 9th Grade Math Score":ninethgrademathscore, "Avg 10th Grade Math Score":tenthgrademathscore,
                                "Avg 11th Grade Math Score":eleventhgrademathscore, "Avg 12th Grade Math Score":twelvethgrademathscore})

mathsummarydata = mathsummarydata.rename_axis("School Name")
mathsummarydata["Avg 9th Grade Math Score"] = mathsummarydata["Avg 9th Grade Math Score"].astype(float).map("{:.2f}".format)
mathsummarydata["Avg 10th Grade Math Score"] = mathsummarydata["Avg 10th Grade Math Score"].astype(float).map("{:.2f}".format)
mathsummarydata["Avg 11th Grade Math Score"] = mathsummarydata["Avg 11th Grade Math Score"].astype(float).map("{:.2f}".format)
mathsummarydata["Avg 12th Grade Math Score"] = mathsummarydata["Avg 12th Grade Math Score"].astype(float).map("{:.2f}".format)

mathsummarydata


,Avg 9th Grade Math Score,Avg 10th Grade Math Score,Avg 11th Grade Math Score,Avg 12th Grade Math Score
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
# collect average scores for each grade grouped by school_name
ninethgradereadscore = ninethgrade.groupby(['school_name']).mean()["math_score"]
tenthgradereadscore = tenthgrade.groupby(['school_name']).mean()["math_score"]
eleventhgradereadscore = eleventhgrade.groupby(['school_name']).mean()["math_score"]
twelvethgradereadscore = twelvethgrade.groupby(['school_name']).mean()["math_score"]

# create DataFrame for summary data
readsummarydata = pd.DataFrame({"Avg 9th Grade Reading Score":ninethgradereadscore, "Avg 10th Grade Reading Score":tenthgradereadscore,
                                "Avg 11th Grade Reading Score":eleventhgradereadscore, "Avg 12th Grade Reading Score":twelvethgradereadscore})

readsummarydata = readsummarydata.rename_axis("School Name")
readsummarydata["Avg 9th Grade Reading Score"] = readsummarydata["Avg 9th Grade Reading Score"].astype(float).map("{:.2f}".format)
readsummarydata["Avg 10th Grade Reading Score"] = readsummarydata["Avg 10th Grade Reading Score"].astype(float).map("{:.2f}".format)
readsummarydata["Avg 11th Grade Reading Score"] = readsummarydata["Avg 11th Grade Reading Score"].astype(float).map("{:.2f}".format)
readsummarydata["Avg 12th Grade Reading Score"] = readsummarydata["Avg 12th Grade Reading Score"].astype(float).map("{:.2f}".format)

readsummarydata

,Avg 9th Grade Reading Score,Avg 10th Grade Reading Score,Avg 11th Grade Reading Score,Avg 12th Grade Reading Score
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ['<$585', '$585-615', '$615-645', '$645-675']

In [9]:
# create initial DataFrame based on school summary data
schoolspendbin = schoolsummaryfile.copy()

# create bin column
schoolspendbin["Spending Ranges (Per Student)"] = pd.cut(schoolsummaryfile["Per Student Budget"], spending_bins, labels=group_names)

# collect required data into bins

spendavgmathbin = schoolspendbin.groupby(['Spending Ranges (Per Student)']).mean()["Average Math Score"]

spendavgreadbin = schoolspendbin.groupby(['Spending Ranges (Per Student)']).mean()["Average Reading Score"]

spendmathpercentbin = schoolspendbin.groupby(['Spending Ranges (Per Student)']).mean()["% Passing Math"]

spendreadpercentbin = schoolspendbin.groupby(['Spending Ranges (Per Student)']).mean()["% Passing Reading"]

spendallpercentbin = schoolspendbin.groupby(['Spending Ranges (Per Student)']).mean()["% Overall Passing Rate"]


# Create dataframe
spendscoreschool = pd.DataFrame({"Average Math Score":spendavgmathbin,
                                "Average Reading Score":spendavgreadbin,
                                "% Passing Math":spendmathpercentbin,
                                "% Passing Reading":spendreadpercentbin,
                                "% Overall Passing Rate":spendallpercentbin})

#Format dataframe
spendscoreschooldisplay = spendscoreschool.copy()
spendscoreschooldisplay["Average Math Score"] = spendscoreschooldisplay["Average Math Score"].astype(float).map("{:,.2f}".format)
spendscoreschooldisplay["Average Reading Score"] = spendscoreschooldisplay["Average Reading Score"].astype(float).map("{:,.2f}".format)
spendscoreschooldisplay["% Passing Math"] = spendscoreschooldisplay["% Passing Math"].astype(float).map("{:,.1f}%".format)
spendscoreschooldisplay["% Passing Reading"] = spendscoreschooldisplay["% Passing Reading"].astype(float).map("{:.2f}%".format)
spendscoreschooldisplay["% Overall Passing Rate"] = spendscoreschooldisplay["% Overall Passing Rate"].astype(float).map("{:.2f}%".format)

spendscoreschooldisplay

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.5%,96.61%,95.04%
$585-615,83.60,83.89,94.2%,95.90%,95.07%
$615-645,79.08,81.89,75.7%,86.11%,80.89%
$645-675,77.00,81.03,66.2%,81.13%,73.65%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [10]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [11]:
# create initial DataFrame based on school summary data
schoolsizebin = schoolsummaryfile.copy()
#schoolsizebin.rename(columns={'Total Students':'School Size'})
# create bin column
schoolsizebin["School Size"] = pd.cut(schoolsummaryfile["Total Students"], size_bins, labels=group_names)

# collect required data into bins

sizeavgmathbin = schoolsizebin.groupby(['School Size']).mean()["Average Math Score"]

sizeavgreadbin = schoolsizebin.groupby(['School Size']).mean()["Average Reading Score"]

sizemathpercentbin = schoolsizebin.groupby(['School Size']).mean()["% Passing Math"]

sizereadpercentbin = schoolsizebin.groupby(['School Size']).mean()["% Passing Reading"]

sizeallpercentbin = schoolsizebin.groupby(['School Size']).mean()["% Overall Passing Rate"]


# Create dataframe
sizescoreschool = pd.DataFrame({"Average Math Score":sizeavgmathbin,
                                "Average Reading Score":sizeavgreadbin,
                                "% Passing Math":sizemathpercentbin,
                                "% Passing Reading":sizereadpercentbin,
                                "% Overall Passing Rate":sizeallpercentbin})

#Format dataframe
sizescoreschooldisplay = sizescoreschool.copy()
sizescoreschooldisplay["Average Math Score"] = sizescoreschooldisplay["Average Math Score"].astype(float).map("{:,.2f}".format)
sizescoreschooldisplay["Average Reading Score"] = sizescoreschooldisplay["Average Reading Score"].astype(float).map("{:,.2f}".format)
sizescoreschooldisplay["% Passing Math"] = sizescoreschooldisplay["% Passing Math"].astype(float).map("{:,.1f}%".format)
sizescoreschooldisplay["% Passing Reading"] = sizescoreschooldisplay["% Passing Reading"].astype(float).map("{:.2f}%".format)
sizescoreschooldisplay["% Overall Passing Rate"] = sizescoreschooldisplay["% Overall Passing Rate"].astype(float).map("{:.2f}%".format)

sizescoreschooldisplay

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.82,83.93,93.6%,96.10%,94.82%
Medium (1000-2000),83.37,83.86,93.6%,96.79%,95.20%
Large (2000-5000),77.75,81.34,70.0%,82.77%,76.36%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [12]:
schooltypestats = pycityschoolsdata.copy()

# collect needed values grouped by school
typestudtotal = schooltypestats.groupby(['type']).count()["Student ID"]

typeavgmathscore = schooltypestats.groupby(['type']).mean()["math_score"]

typeavgreadscore = schooltypestats.groupby(['type']).mean()["reading_score"]

typepassmath = schooltypestats.loc[schooltypestats["math_score"] >=70].groupby(['type']).count()["math_score"]
typepassmathrate = typepassmath / typestudtotal *100

typepassread = schooltypestats.loc[schooltypestats["reading_score"] >=70].groupby(['type']).count()["reading_score"]
typepassreadrate = typepassread / typestudtotal *100

typeallpassrate = (typepassmathrate + typepassreadrate)/2

# #create sumary DataFrame
typesummaryfile = pd.DataFrame({"Average Math Score":typeavgmathscore, "Average Reading Score":typeavgreadscore, 
                                "% Passing Math":typepassmathrate,"% Passing Reading":typepassreadrate, 
                                "% Overall Passing Rate":typeallpassrate})


# format DataFrame values
typesummarydata = typesummaryfile.copy()
typesummarydata = typesummarydata.rename_axis("Type")
typesummarydata['Average Math Score'] = typesummarydata['Average Math Score'].astype(float).map('{:.2f}'.format)
typesummarydata['Average Reading Score'] = typesummarydata['Average Reading Score'].astype(float).map('{:.2f}'.format)
typesummarydata['% Passing Math'] = typesummarydata['% Passing Math'].astype(float).map('{:.2f}%'.format)
typesummarydata['% Passing Reading'] = typesummarydata['% Passing Reading'].astype(float).map('{:.2f}%'.format)
typesummarydata['% Overall Passing Rate'] = typesummarydata['% Overall Passing Rate'].astype(float).map('{:.2f}%'.format)


typesummarydata


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Type,,,,,
Charter,83.41,83.90,93.70%,96.65%,95.17%
District,76.99,80.96,66.52%,80.91%,73.71%
